<a href="https://colab.research.google.com/github/omkar806/on-device-experiments/blob/main/Assignment_Omkar_Malpure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Run the below cell to install all the libraries

In [1]:
!pip install -q langchain openai langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 5.4 MB/s eta 0:00:00


#Import all the neccessary libraries using the below cell

In [2]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from datetime import datetime
from langchain.callbacks import get_openai_callback
import os

#Defining a Pydantic Model to extract the exact information for Each key and Validate the json output given by the LLM.

In [4]:
#Defining a pydantic model for getting the exact strcutured json from the input query as we want
class Candidate(BaseModel):
    entity: str = Field(description="The company name mentioned in the query (e.g., Flipkart, Amazon).")
    parameter: str = Field(description="The performance metric mentioned in the query (e.g., GMV, revenue, profit)")
    startDate: str = Field(description=f"The start date of the time period for which the metric is requested. By default If the user query does not explicitly mention the start date.Assume start date to be : Today's date(is {datetime.today().date()}) minus one year.")
    endDate: str = Field(description=f"The end date of the time period for which the metric is requested.By default If the user query does not explicitly mention the start date. Assume the end date to be : Today's date(is {datetime.today().date()}).")

#Initialising the OpenAI model , Please provide your valid OPENAI_API_KEY in the cell below in the first line

In [5]:
os.environ["OPENAI_API_KEY"] = "Enter your OPENAI_API_KEY"
model_name = "gpt-3.5-turbo-instruct"
temperature = 0.0
model = OpenAI(model_name=model_name, temperature=temperature, max_tokens=800)

#Function responsible for taking the user query processing it and Validating it with Pydantic model keys and printing the Json output

In [ ]:
def bot_response(user_query):
  # user_query="Get me Flipkart's GMV for last one year ?"
  parser = PydanticOutputParser(pydantic_object=Candidate)
  prompt = PromptTemplate(
    template="Answer the user query.\n {format_instructions}\n {query} \n Always Strictly return a Json having the keys mentioned entity,parameter,startDate , endDate by analysing their description.",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
  )
  input = prompt.format_prompt(query=str(user_query))
  with get_openai_callback() as cb:
      result = model(input.to_string())
      class_object= parser.parse(result)
      dict_object=class_object.__dict__
      print("Bot: ",dict_object)

#Below cell can be used for interaction , You can test your prompts here

In [8]:
while True:
  user_query = input("User: ")
  if user_query == "exit":
    break
  bot_response(user_query)

User: Get me Flipkart's GMV for last one year ?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Bot:  {'entity': 'Flipkart', 'parameter': 'GMV', 'startDate': '2023-05-05', 'endDate': '2024-05-05'}


KeyboardInterrupt: Interrupted by user